<a href="https://colab.research.google.com/github/daemon-Lee/simplex_method_for_linear_program/blob/master/project/game_theory/Game_theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Copyright 2020 Duy L.Dinh. { display-mode: "form" }
#@markdown CS1302 HE130655.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
#@title install lib
!pip install cplex pulp

     |████████████████████████████████| 31.0MB 132kB/s 
     |████████████████████████████████| 40.6MB 103kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for amply: filename=amply-0.1.2-cp36-none-any.whl size=16572 sha256=89301305987fc4eb3fda13cad2995c3727e24ae0c5919b3b3ed16d5635e8b026
  Stored in directory: /root/.cache/pip/wheels/84/18/f7/e5c3ed13ed5bb721763f77d4a924331d59ef115ce61c9d26eb
Successfully built amply


In [ ]:
import numpy as np

In [ ]:
#@title Implement cplex lib
import cplex
def cplex_lib(A, b, c):
    A = A.astype(int).tolist()
    b = b.astype(int).tolist()
    c = c.astype(int).tolist()
    # Input all the data and parameters here
    num_constraints = len(A)
    num_decision_var = len(c)

    # constraint_type = ["L", "G", "E"] # Less, Greater, Equal
    constraint_type = ["L"]*(num_constraints-1)
    constraint_type.append('E')

    # ============================================================

    # Establish the Linear Programming Model
    myProblem = cplex.Cplex()

    # Add the decision variables and set their lower bound and upper bound (if necessary)
    myProblem.variables.add(names= ["x"+str(i) for i in range(num_decision_var)])
    for i in range(num_decision_var):
        myProblem.variables.set_lower_bounds(i, 0.0)

    # Add constraints
    for i in range(num_constraints):
        myProblem.linear_constraints.add(
            lin_expr= [cplex.SparsePair(ind= [j for j in range(num_decision_var)], val= A[i])],
            rhs= [b[i]],
            names = ["c"+str(i)],
            senses = [constraint_type[i]]
        )

    # Add objective function and set its sense
    for i in range(num_decision_var):
        myProblem.objective.set_linear([(i, c[i])])
    myProblem.objective.set_sense(myProblem.objective.sense.maximize)

    # Solve the model and print the answer
    myProblem.solve()
    return{
        'objective': myProblem.solution.get_objective_value(),
        'status': myProblem.solution.get_status_string(),
        'sol': myProblem.solution.get_values()
    }

In [ ]:
#@title Implement pulp lib
import pulp as p
def pulp_lib(A, b, c):
    # Generate B and N
    n_contrain = len(A)
    n_var = len(c)

    # Create a LP Minimization problem 
    Lp_prob = p.LpProblem('Problem', p.LpMaximize) 

    # Create problem Variables
    x = [p.LpVariable("x"+str(i), lowBound = 0) for i in range(1,n_var)]
    v = p.LpVariable("v")

    # Objective Function 
    Lp_prob += v

    # Constraints:
    for i in range(n_contrain-1):
        contrain = 0
        for j in range(n_var-1):
            contrain += A[i,j]*x[j] + (1/(n_contrain-1))*v <= b[i]
        Lp_prob += contrain

    contrain = 0
    for j in range(n_var-1):
        contrain += A[n_contrain-1,j]*x[j] + (1/(n_contrain-1))*v == (1/(n_contrain-1))
    Lp_prob += contrain
    
    # Display the problem 
    print(Lp_prob)
    
    status = Lp_prob.solve() # Solver 
    
    value = [p.value(x[i]) for i in range(0,n_var-1)]
    
    return {
        'value': value,
        'status': p.LpStatus[status],
        'objective': p.value(Lp_prob.objective)
    }

In [ ]:
len_number = 100

In [ ]:
A = np.zeros((len_number,len_number))

Problem: Players A and B each pick a number between 1 and 100. The game is a draw if both players pick the same number. Otherwise, the player who picks the smaller number wins unless that smaller number is one less than the opponent’s number, in which case the opponent wins. Find the optimal strategy for this game.

In [ ]:
#@markdown add contrain follow problem
for i in range(len_number):
    for j in range(len_number):
        if (i != j) and (i < j-1):
            A[i,j] = -1
            A[j,i] = 1
        elif (i==j-1):
            A[i,j] = 1
            A[j,i] = -1

In [ ]:
e = np.ones(len_number)[np.newaxis]

In [ ]:
#@markdown [[-A, e.T],[e, 0]] * [[x], [v]]

a1 = np.concatenate((-A,e.T),axis=1)
a2 = np.concatenate((e,np.array([0])[np.newaxis]),axis=1)
A = np.concatenate((a1,a2))

In [ ]:
A

array([[-0., -1.,  1., ...,  1.,  1.,  1.],
       [ 1., -0., -1., ...,  1.,  1.,  1.],
       [-1.,  1., -0., ...,  1.,  1.,  1.],
       ...,
       [-1., -1., -1., ..., -0., -1.,  1.],
       [-1., -1., -1., ...,  1., -0.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  0.]])

In [ ]:
b = c = np.concatenate((np.zeros(len_number),np.array([1])))

In [ ]:
cplex_lib(A, b, c)

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.01 sec. (1.51 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual infeasibility =             1.000000
Iteration:     3   Dual objective     =             0.000000


{'objective': 1.1102230246251565e-16,
 'sol': [0.3333333333333334,
  0.3333333333333333,
  0.3333333333333333,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'status': 'optimal'}

In [ ]:
pulp_lib(A, b, c)

Problem:
MAXIMIZE
1*v + 0
SUBJECT TO
_C1: 1 v + x10 + x100 + x11 + x12 + x13 + x14 + x15 + x16 + x17 + x18 + x19
 - x2 + x20 + x21 + x22 + x23 + x24 + x25 + x26 + x27 + x28 + x29 + x3 + x30
 + x31 + x32 + x33 + x34 + x35 + x36 + x37 + x38 + x39 + x4 + x40 + x41 + x42
 + x43 + x44 + x45 + x46 + x47 + x48 + x49 + x5 + x50 + x51 + x52 + x53 + x54
 + x55 + x56 + x57 + x58 + x59 + x6 + x60 + x61 + x62 + x63 + x64 + x65 + x66
 + x67 + x68 + x69 + x7 + x70 + x71 + x72 + x73 + x74 + x75 + x76 + x77 + x78
 + x79 + x8 + x80 + x81 + x82 + x83 + x84 + x85 + x86 + x87 + x88 + x89 + x9
 + x90 + x91 + x92 + x93 + x94 + x95 + x96 + x97 + x98 + x99 <= 0

_C2: 1 v + x1 + x10 + x100 + x11 + x12 + x13 + x14 + x15 + x16 + x17 + x18
 + x19 + x20 + x21 + x22 + x23 + x24 + x25 + x26 + x27 + x28 + x29 - x3 + x30
 + x31 + x32 + x33 + x34 + x35 + x36 + x37 + x38 + x39 + x4 + x40 + x41 + x42
 + x43 + x44 + x45 + x46 + x47 + x48 + x49 + x5 + x50 + x51 + x52 + x53 + x54
 + x55 + x56 + x57 + x58 + x59 + x6 + x60 + x

{'objective': 0.0,
 'status': 'Optimal',
 'value': [0.33333333,
  0.33333333,
  0.33333333,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]}

Player want to win should choose number in [1,2,3]

Make another example

In [ ]:
A = np.array([[2,-6],
              [-9,4]])

In [ ]:
#@markdown [[-A, e.T],[e, 0]] * [[x], [v]]
e = np.ones(len(A))[np.newaxis]
a1 = np.concatenate((-A,e.T),axis=1)
a2 = np.concatenate((e,np.array([0])[np.newaxis]),axis=1)
A = np.concatenate((a1,a2))

In [ ]:
A

array([[-2.,  6.,  1.],
       [ 9., -4.,  1.],
       [ 1.,  1.,  0.]])

In [ ]:
b = c = np.concatenate((np.zeros(len(A)),np.array([1])))

In [ ]:
pulp_lib(A, b, c)

Problem:
MAXIMIZE
1*v + 0
SUBJECT TO
_C1: 1.5 v - 2 x1 + 6 x2 + x3 <= 0

_C2: 1.5 v + 9 x1 - 4 x2 + x3 <= 0

_C3: 1.5 v + x1 + x2 = 1.5

VARIABLES
v free Continuous
x1 Continuous
x2 Continuous
x3 Continuous



{'objective': -21333.769,
 'status': 'Infeasible',
 'value': [20001.533, 12000.62, 0.0]}